In [1]:
import glob 
import os
from tqdm import tqdm

covers = []

In [2]:
from pathlib import Path

for path in tqdm(Path('/volumes/G-DRIVE USB-C').rglob('*.jpg')):
    absolute = os.path.abspath(path)
    covers.append(absolute)

0it [00:00, ?it/s]


In [3]:
covers

[]

In [4]:
#len(os.listdir('/volumes/Multimedia/FLAC/'))

# Setup

In [5]:
import requests
import musicbrainzngs
import pandas as pd
from tqdm.autonotebook import tqdm


#!pip install musicbrainzngs
#!pip install pyvis
auth = musicbrainzngs.auth("username","password")

(musicbrainzngs.set_useragent("app","Client_ID", 
                              contact="email"))

/Users/adamgoldstein/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  after removing the cwd from sys.path.


# Get Results for Artist ID Search

In [23]:
#Need to modify to label the purpose: Producer, Performer, etc

def get_artist_works(artist_id):

    result_artist = (musicbrainzngs.get_artist_by_id(artist_id
                                                     ,includes=['work-rels']))

    df_artist_work_relation = pd.DataFrame()

    df_artist_interate = pd.DataFrame(result_artist['artist']['work-relation-list'])

    #Work ID - Loop by length of dataframe - work[0] on outer to get id
    for x in range(len(df_artist_interate)):
        df_iter = pd.DataFrame(result_artist['artist']['work-relation-list'][x])
        df_artist_work_relation = pd.concat([df_artist_work_relation,df_iter])
        
    
        product = df_artist_work_relation.reset_index()
    
        return product


def establish_work_to_artist_relationships(work_ids,work_names):
    
    artist_names = pd.DataFrame()
    
    for ident in tqdm(work_ids):
    
        result_work = (pd.DataFrame(musicbrainzngs.get_work_by_id(ident
                                                                  ,includes=['artist-rels'])))

        works = pd.DataFrame(result_work['work']['artist-relation-list'])
        
        works['work_id'] = ident
        
        artist_names = pd.concat([artist_names,works])
        
    return artist_names.reset_index()
        

def rename_columns(df,column_name):

    for col in df.columns:
        df = df.rename(columns={col:f'{column_name}_{col}'})
    return df

In [24]:
David_Byrne = 'd4659efb-b8eb-4f03-95e9-f69ce35967a9'



In [25]:
product = get_artist_works(David_Byrne)

In [26]:
product = rename_columns(product,'Work')

In [27]:
product_ids = product[product['Work_index']=='id']

In [28]:
work_artist_relationships = (establish_work_to_artist_relationships(product.query('Work_index == "id"')['Work_work'].to_list()
                                            ,product.query('Work_index == "title"')['Work_work'].to_list()))


In [29]:
for key in work_artist_relationships['artist'][0].keys():
    work_artist_relationships[key] = work_artist_relationships['artist'].apply(lambda x: x[key])

In [30]:
work_artist_relationships = rename_columns(work_artist_relationships,'Artist')

In [31]:
to_graph = work_artist_relationships.merge(product_ids,left_on='Artist_work_id',right_on='Work_work')

In [33]:
to_graph

,Artist_index,Artist_type,Artist_type-id,Artist_target,Artist_direction,Artist_artist,Artist_target-credit,Artist_work_id,Artist_id,Artist_name,Artist_sort-name,Work_index,Work_type,Work_type-id,Work_target,Work_direction,Work_work
0,0,Person,d3fd781c-5894-47e2-8c12-86cc0e2c8d08,e0d6cbe7-60cc-4d3d-b5a7-36db7b8b4931,backward,"{'id': 'e0d6cbe7-60cc-4d3d-b5a7-36db7b8b4931',...",NaN,f4cf3653-f472-3de8-8a80-7b9fdb661ba8,e0d6cbe7-60cc-4d3d-b5a7-36db7b8b4931,Ashley Beedle,"Beedle, Ashley",id,arranger,d3fd781c-5894-47e2-8c12-86cc0e2c8d08,f4cf3653-f472-3de8-8a80-7b9fdb661ba8,forward,f4cf3653-f472-3de8-8a80-7b9fdb661ba8
1,1,Person,d3fd781c-5894-47e2-8c12-86cc0e2c8d08,d4659efb-b8eb-4f03-95e9-f69ce35967a9,backward,"{'id': 'd4659efb-b8eb-4f03-95e9-f69ce35967a9',...",NaN,f4cf3653-f472-3de8-8a80-7b9fdb661ba8,d4659efb-b8eb-4f03-95e9-f69ce35967a9,David Byrne,"Byrne, David",id,arranger,d3fd781c-5894-47e2-8c12-86cc0e2c8d08,f4cf3653-f472-3de8-8a80-7b9fdb661ba8,forward,f4cf3653-f472-3de8-8a80-7b9fdb661ba8
2,2,Person,d3fd781c-5894-47e2-8c12-86cc0e2c8d08,114d409c-2a82-49f1-9df4-e937e50a5da4,backward,"{'id': '114d409c-2a82-49f1-9df4-e937e50a5da4',...",NaN,f4cf3653-f472-3de8-8a80-7b9fdb661ba8,114d409c-2a82-49f1-9df4-e937e50a5da4,Darren House,"House, Darren",id,arranger,d3fd781c-5894-47e2-8c12-86cc0e2c8d08,f4cf3653-f472-3de8-8a80-7b9fdb661ba8,forward,f4cf3653-f472-3de8-8a80-7b9fdb661ba8
3,3,Person,d3fd781c-5894-47e2-8c12-86cc0e2c8d08,33f8dbe6-9642-4d23-804f-1ed06987e34c,backward,"{'id': '33f8dbe6-9642-4d23-804f-1ed06987e34c',...",Darren Rock,f4cf3653-f472-3de8-8a80-7b9fdb661ba8,33f8dbe6-9642-4d23-804f-1ed06987e34c,Rocky,Rocky,id,arranger,d3fd781c-5894-47e2-8c12-86cc0e2c8d08,f4cf3653-f472-3de8-8a80-7b9fdb661ba8,forward,f4cf3653-f472-3de8-8a80-7b9fdb661ba8
4,4,Person,a255bca1-b157-4518-9108-7b147dc3fc68,e0d6cbe7-60cc-4d3d-b5a7-36db7b8b4931,backward,"{'id': 'e0d6cbe7-60cc-4d3d-b5a7-36db7b8b4931',...",NaN,f4cf3653-f472-3de8-8a80-7b9fdb661ba8,e0d6cbe7-60cc-4d3d-b5a7-36db7b8b4931,Ashley Beedle,"Beedle, Ashley",id,arranger,d3fd781c-5894-47e2-8c12-86cc0e2c8d08,f4cf3653-f472-3de8-8a80-7b9fdb661ba8,forward,f4cf3653-f472-3de8-8a80-7b9fdb661ba8
5,5,Person,a255bca1-b157-4518-9108-7b147dc3fc68,d4659efb-b8eb-4f03-95e9-f69ce35967a9,backward,"{'id': 'd4659efb-b8eb-4f03-95e9-f69ce35967a9',...",NaN,f4cf3653-f472-3de8-8a80-7b9fdb661ba8,d4659efb-b8eb-4f03-95e9-f69ce35967a9,David Byrne,"Byrne, David",id,arranger,d3fd781c-5894-47e2-8c12-86cc0e2c8d08,f4cf3653-f472-3de8-8a80-7b9fdb661ba8,forward,f4cf3653-f472-3de8-8a80-7b9fdb661ba8
6,6,Person,a255bca1-b157-4518-9108-7b147dc3fc68,114d409c-2a82-49f1-9df4-e937e50a5da4,backward,"{'id': '114d409c-2a82-49f1-9df4-e937e50a5da4',...",NaN,f4cf3653-f472-3de8-8a80-7b9fdb661ba8,114d409c-2a82-49f1-9df4-e937e50a5da4,Darren House,"House, Darren",id,arranger,d3fd781c-5894-47e2-8c12-86cc0e2c8d08,f4cf3653-f472-3de8-8a80-7b9fdb661ba8,forward,f4cf3653-f472-3de8-8a80-7b9fdb661ba8
7,7,Person,a255bca1-b157-4518-9108-7b147dc3fc68,33f8dbe6-9642-4d23-804f-1ed06987e34c,backward,"{'id': '33f8dbe6-9642-4d23-804f-1ed06987e34c',...",Darren Rock,f4cf3653-f472-3de8-8a80-7b9fdb661ba8,33f8dbe6-9642-4d23-804f-1ed06987e34c,Rocky,Rocky,id,arranger,d3fd781c-5894-47e2-8c12-86cc0e2c8d08,f4cf3653-f472-3de8-8a80-7b9fdb661ba8,forward,f4cf3653-f472-3de8-8a80-7b9fdb661ba8


In [32]:
to_graph.query('Artist_name == "Ashley Beedle"')

,Artist_index,Artist_type,Artist_type-id,Artist_target,Artist_direction,Artist_artist,Artist_target-credit,Artist_work_id,Artist_id,Artist_name,Artist_sort-name,Work_index,Work_type,Work_type-id,Work_target,Work_direction,Work_work
0,0,Person,d3fd781c-5894-47e2-8c12-86cc0e2c8d08,e0d6cbe7-60cc-4d3d-b5a7-36db7b8b4931,backward,"{'id': 'e0d6cbe7-60cc-4d3d-b5a7-36db7b8b4931',...",NaN,f4cf3653-f472-3de8-8a80-7b9fdb661ba8,e0d6cbe7-60cc-4d3d-b5a7-36db7b8b4931,Ashley Beedle,"Beedle, Ashley",id,arranger,d3fd781c-5894-47e2-8c12-86cc0e2c8d08,f4cf3653-f472-3de8-8a80-7b9fdb661ba8,forward,f4cf3653-f472-3de8-8a80-7b9fdb661ba8
4,4,Person,a255bca1-b157-4518-9108-7b147dc3fc68,e0d6cbe7-60cc-4d3d-b5a7-36db7b8b4931,backward,"{'id': 'e0d6cbe7-60cc-4d3d-b5a7-36db7b8b4931',...",NaN,f4cf3653-f472-3de8-8a80-7b9fdb661ba8,e0d6cbe7-60cc-4d3d-b5a7-36db7b8b4931,Ashley Beedle,"Beedle, Ashley",id,arranger,d3fd781c-5894-47e2-8c12-86cc0e2c8d08,f4cf3653-f472-3de8-8a80-7b9fdb661ba8,forward,f4cf3653-f472-3de8-8a80-7b9fdb661ba8


# Don't Go Further

from pyvis.network import Network
import pandas as pd

got_net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white")

# set the physics layout of the network
got_net.barnes_hut()
got_data = pd.read_csv("https://www.macalester.edu/~abeverid/data/stormofswords.csv")

sources = got_data['Source']
targets = got_data['Target']
weights = got_data['Weight']



edge_data = zip(sources, targets, weights)

for e in edge_data:
    src = e[0]
    dst = e[1]
    w = e[2]

    got_net.add_node(src, src, title=src)
    got_net.add_node(dst, dst, title=dst)
    got_net.add_edge(src, dst, value=w)

neighbor_map = got_net.get_adj_list()

print(neighbor_map)

# add neighbor data to node hover data
for node in got_net.nodes:
    node["title"] += " Neighbors:<br>" + "<br>".join(neighbor_map[node["id"]])
    node["value"] = len(neighbor_map[node["id"]])

got_net.show("gameofthrones.html")

In [ ]:
#Neighbors
#for i in g.neighbors('David Byrne'):
#    print(i)

In [ ]:
#Central Theme
#nx.degree_centrality(g)

In [ ]:
#Shortest Path
#centrality.betweenness_centrality(g)

# Artist Specific

In [ ]:
result = musicbrainzngs.get_artist_by_id(artist_id,
              includes=['release-groups'],release_type=['album'])


#for release_group in result["artist"]["release-group-list"]:
#    print("{title} ({type})".format(title=release_group["title"],
#                                    type=release_group["type"]))

In [ ]:
#result['artist']

In [ ]:
import pandas as pd
df = pd.DataFrame(result["artist"]["release-group-list"])
df["first-release-date"] = pd.to_datetime(df["first-release-date"])  

df["year"] = pd.DatetimeIndex(df["first-release-date"]).year
df['month'] = pd.DatetimeIndex(df["first-release-date"]).month
df["day"] = pd.DatetimeIndex(df["first-release-date"]).day

In [ ]:
df

In [ ]:
_ = df.groupby("year")["id"].count().plot(kind="bar",title="Annual: Artist Releases")

# Get Album Cover Art

In [ ]:
release_id = "0dd5a352-6ae0-3e0d-bf18-bec30e27807d"
#"7e643344-946e-4118-b934-6574695e68bf"
data = musicbrainzngs.get_release_group_image_list(release_id)


for image in data["images"]:
    if "Front" in image["types"] and image["approved"]:
        print("%s is an approved front image!" % image["thumbnails"]["large"])
        image_link = image["thumbnails"]["large"]
        break
        


# Track Specific

In [ ]:
#musicbrainzngs.get_release_group_by_id(release_id, includes=['releases'])

In [ ]:
#musicbrainzngs.get_release_by_id('f7345598-ef59-44e1-88d0-4b428977fda1',includes=['recordings'])

In [ ]:
#musicbrainzngs.get_recording_by_id('ae077045-d684-466d-823e-b6f229733a5b',includes=['artist-credits'])

In [ ]:
#response = requests.get(image_link, stream=True)
#with open('img.png', 'wb') as out_file:
#    shutil.copyfileobj(response.raw, out_file)